# Data Cleaning Project 01

### Import Libraries

In [61]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import os

# Ensure output directory exists
if not os.path.exists('output'):
    os.makedirs('output')

### Import CSV File

In [9]:
df = pd.read_csv("Messy_Data.CSV")
df.head()

,CustomerID,CustomerName,Email,OrderDate,ProductCategory,Price,Rating
0,1102,customer 0,customer0@example.com,2023-12-31,Books,51.18,5.0
1,1435,customer 1,customer1@example.com,2023-01-05,Books,55.38,5.0
2,1860,customer 2,customer2@example.com,04/29/23,Clothing,117.75,3.0
3,1270,CUSTOMER 3,customer3@example.com,10/16/23,Clothing,88.94,3.0
4,1106,customer 4,customer4@example.com,2023-03-06,Clothing,57.99,5.0


### Data Understanding

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerID       1020 non-null   int64  
 1   CustomerName     1020 non-null   object 
 2   Email            972 non-null    object 
 3   OrderDate        1020 non-null   object 
 4   ProductCategory  1020 non-null   object 
 5   Price            1000 non-null   float64
 6   Rating           973 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 55.9+ KB


### Identify and Handle Missing Data

In [13]:
print("Missing values before cleaning:")
print(df.isnull().sum())

Missing values before cleaning:
CustomerID          0
CustomerName        0
Email              48
OrderDate           0
ProductCategory     0
Price              20
Rating             47
dtype: int64


`Insights:` Dataset includes -
- Rows/Columns : 1020/7
- Email column have 48 null values
- Price column have 20 null values
- Rating column have 47 null values

In [22]:
# Impute Email with placeholder

df['Email'] = df['Email'].fillna('unknown@example.com')

In [25]:
# Impute Rating with median

df['Rating'] = df['Rating'].fillna(df['Rating'].median())

In [27]:
# Drop rows with missing Price (critical for analysis)

df.dropna(subset=['Price'], inplace=True)

In [29]:
print("\nMissing values after cleaning:")
print(df.isnull().sum())


Missing values after cleaning:
CustomerID         0
CustomerName       0
Email              0
OrderDate          0
ProductCategory    0
Price              0
Rating             0
dtype: int64


### Remove Duplicates and Handle Outliers

In [37]:
# Remove duplicates based on CustomerID and OrderDate

df.drop_duplicates(subset=['CustomerID', 'OrderDate'], keep='first', inplace=True)
print(f"Number of rows after removing duplicates: {len(df)}")

Number of rows after removing duplicates: 987


In [39]:
# Handle outliers in Price using IQR

Q1 = df['Price'].quantile(0.25)
Q3 = df['Price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Cap outliers

df['Price'] = df['Price'].clip(lower=0, upper=upper_bound)
print(f"Price outliers capped. New min: {df['Price'].min()}, max: {df['Price'].max()}")

Price outliers capped. New min: 0.0, max: 236.40750000000003


### Standardize Data Entries

In [42]:
# Standardize CustomerName to title case

df['CustomerName'] = df['CustomerName'].str.title()

In [44]:
# Validate and standardize Email

def validate_email(email):
    pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return bool(re.match(pattern, email))

df['Email'] = df['Email'].apply(lambda x: x if validate_email(x) else 'invalid@example.com')

In [46]:
# Standardize OrderDate to YYYY-MM-DD

df['OrderDate'] = pd.to_datetime(df['OrderDate'], errors='coerce').dt.strftime('%Y-%m-%d')

In [48]:
# Correct ProductCategory misspellings

category_mapping = {'Electornics': 'Electronics', 'Clothng': 'Clothing'}
df['ProductCategory'] = df['ProductCategory'].replace(category_mapping)

### Save the cleaned dataset

In [51]:
df.to_csv('Messy_Data_Cleaned.csv', index=False)
print("\nCleaned dataset saved as 'Messy_Data_Cleaned.csv'")
print("\nSample of cleaned dataset:")
df.head()


Cleaned dataset saved as 'Messy_Data_Cleaned.csv'

Sample of cleaned dataset:


,CustomerID,CustomerName,Email,OrderDate,ProductCategory,Price,Rating
0,1102,Customer 0,customer0@example.com,2023-12-31,Books,51.18,5.0
1,1435,Customer 1,customer1@example.com,2023-01-05,Books,55.38,5.0
2,1860,Customer 2,customer2@example.com,NaN,Clothing,117.75,3.0
3,1270,Customer 3,customer3@example.com,NaN,Clothing,88.94,3.0
4,1106,Customer 4,customer4@example.com,2023-03-06,Clothing,57.99,5.0


In [55]:
# Summary statistics of cleaned data

print("\nSummary statistics of cleaned data:")
df.describe(include='all')


Summary statistics of cleaned data:


,CustomerID,CustomerName,Email,OrderDate,ProductCategory,Price,Rating
count,987.000000,987,987,679,987,987.000000,987.000000
unique,NaN,980,934,310,3,NaN,NaN
top,NaN,Customer 811,unknown@example.com,2023-04-08,Clothing,NaN,NaN
freq,NaN,2,47,6,412,NaN,NaN
mean,1501.087133,NaN,NaN,NaN,NaN,98.880426,3.205674
std,289.453672,NaN,NaN,NaN,NaN,49.775098,1.318171
min,1000.000000,NaN,NaN,NaN,NaN,0.000000,1.000000
25%,1240.500000,NaN,NaN,NaN,NaN,64.345000,2.000000
50%,1502.000000,NaN,NaN,NaN,NaN,100.290000,3.000000
75%,1750.500000,NaN,NaN,NaN,NaN,133.170000,4.000000
